##### Compare our code with Dr. Adams solution

In [1]:
# import pickle file and print the content
import pickle

# variableAndQmatrix.pickle
with open('variableAndQmatrix.pickle', 'rb') as f:
    data = pickle.load(f)


In [2]:
import numpy as np
# get the determinant of each Q matrix
for variable, Qmatrix in data.items():
    print(variable, sep='\n')
    print('Determinant:', (np.linalg.det(Qmatrix), 2))


EP_DISABL
Determinant: (array([0.]), 2)
EP_PCI
Determinant: (array([0.]), 2)
EP_LIMENG
Determinant: (array([-1.17615891e-10]), 2)
EP_CROWD
Determinant: (array([0.]), 2)
EP_UNINSUR
Determinant: (array([3.91085014e-15]), 2)


In [14]:
Qmatrix = data['EP_DISABL'][0]

In [15]:
# ger the shape of the df
print(Qmatrix.shape)

(11, 11)


##### Dr. Adams code

In [16]:
import scipy as sp
from scipy.linalg import solve
from scipy.sparse.linalg import spsolve

In [17]:
# Qmatrix dataframe to numpy array
Qmatrix_ = Qmatrix.to_numpy()

In [18]:
n = Qmatrix_.shape[0]
n

11

In [19]:
# Add jitter to the diagonal of 
# In the context of the provided code, "jittered" refers to the process of adding 
# small random variations to the diagonal elements of the matrix Qmatrix_. 
# This is achieved by multiplying a sparse diagonal matrix (constructed with ones along the main diagonal)
# by the maximum value along the diagonal of Qmatrix_ and the square root of the machine epsilon.

Q_jitter = Qmatrix_ + sp.sparse.diags(np.ones(n)) * max(Qmatrix_.diagonal()) * np.sqrt(

    np.finfo(np.float64).eps

)

In [20]:
# inverse of precision (Q) is cov

Q_perturbed = sp.sparse.csc_array(Q_jitter)

b = sp.sparse.identity(n, format='csc')

sigma = spsolve(Q_perturbed, b)

In [21]:
cov = sigma.toarray()
Q_jitter.dot(cov)

matrix([[ 1.00000000e+00,  0.00000000e+00, -4.65661287e-10,
         -4.65661287e-10,  0.00000000e+00, -4.65661287e-10,
         -4.65661287e-10,  0.00000000e+00, -4.65661287e-10,
         -4.65661287e-10, -4.65661287e-10],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00,  

In [22]:
# V \in Null(Q)

V = np.ones(n)  # from pg. 6

W = sigma @ V.T  # \Sigma * B in 3.17

Q_inv = sigma - np.outer(W * solve(V @ W, np.ones(1)), W.T)

In [23]:
Q_inv

array([[  762601.36738122, -2033601.93801671,   762600.99238118,
          762600.24238134, -2033601.93801671,   762600.4923813 ,
          762601.11738119, -2033601.93801671,   762600.4923813 ,
          762600.74238127,   762600.36738132],
       [-2033601.93801671,  5422939.05693343, -2033601.93801671,
        -2033601.93801671,  5422938.05693347, -2033601.93801671,
        -2033601.93801671,  5422938.39026679, -2033601.93801671,
        -2033601.93801671, -2033601.93801671],
       [  762600.99238118, -2033601.93801671,   762602.61738096,
          762599.86738145, -2033601.93801671,   762600.11738138,
          762601.74238105, -2033601.93801671,   762600.11738138,
          762600.36738132,   762599.99238142],
       [  762600.24238134, -2033601.93801671,   762599.86738145,
          762602.1173811 , -2033601.93801671,   762600.8673812 ,
          762599.99238142, -2033601.93801671,   762600.8673812 ,
          762600.61738125,   762601.24238116],
       [-2033601.93801671,  5422

In [24]:
# grabbing diag of cov gives var and

# arithmetic mean in log-space becomes geometric mean after exp

scaling = np.exp(np.sum(np.log(np.diag(Q_inv))) / n)

print(scaling)

1302099.5075113948
